# 🎛️ Orpheus Engine Web Demo

Interactive demonstration of Orpheus Engine DAW components showcasing platform capabilities and cross-platform compatibility.

## Overview

This notebook demonstrates:
- **Platform Detection**: Automatic detection of Electron, Browser, and Python environments
- **Component Showcase**: Interactive DAW widgets and interfaces
- **Capability Testing**: Real-time testing of audio, file system, and notification APIs
- **Cross-Platform Compatibility**: Seamless operation across different platforms

**Compatible Environments**: Jupyter Lab, Jupyter Notebook, VS Code Jupyter, Google Colab

## 🔧 Installation & Setup

### Prerequisites
- **Python 3.8+** (Python 3.9+ recommended)
- **Jupyter Lab or Jupyter Notebook**
- **Git** (for cloning the repository)
- **FFmpeg** (for audio processing)

### Quick Installation

#### Option 1: Install from requirements.txt (Recommended)
```bash
# Navigate to the demo directory
cd demo/

# Install all dependencies including TensorFlow and MLflow
pip install -r requirements.txt

# Install TensorFlow and additional ML packages
pip install tensorflow>=2.13.0 torch>=2.0.0
```

#### Option 2: Manual Installation
```bash
# Core ML and tracking
pip install mlflow>=2.8.0 mlflow-ui>=2.8.0

# TensorFlow and PyTorch for deep learning
pip install tensorflow>=2.13.0 torch>=2.0.0 torchvision torchaudio

# Audio processing libraries
pip install librosa>=0.10.0 soundfile>=0.12.0 scipy>=1.10.0

# Data science stack
pip install numpy>=1.24.0 pandas>=2.0.0 scikit-learn>=1.3.0

# Visualization
pip install matplotlib>=3.6.0 seaborn>=0.12.0 plotly>=5.17.0

# Jupyter ecosystem
pip install jupyter>=1.0.0 ipywidgets>=8.0.0

# Web APIs
pip install fastapi>=0.104.0 uvicorn>=0.24.0 requests>=2.31.0
```

### System-Specific Setup

#### macOS
```bash
# Install FFmpeg and system dependencies
brew install ffmpeg portaudio

# For Apple Silicon Macs
export ARCHFLAGS="-arch arm64"
pip install --no-cache-dir tensorflow librosa soundfile
```

#### Ubuntu/Debian
```bash
# Install system dependencies
sudo apt-get update
sudo apt-get install ffmpeg libsndfile1 portaudio19-dev python3-dev

# Install Python packages
pip install -r requirements.txt
pip install tensorflow torch
```

#### Windows
```bash
# Using conda (recommended for Windows)
conda install -c conda-forge librosa soundfile ffmpeg
pip install tensorflow torch mlflow jupyter plotly
```

### MLflow Setup

```bash
# Start MLflow tracking server (run in separate terminal)
mlflow server --host 127.0.0.1 --port 5000

# Or start with specific backend store
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 127.0.0.1 --port 5000
```

### Verification
Run the next cell to verify your installation.

In [ ]:
# 🔍 Installation Verification & Environment Setup
# Run this cell to verify all dependencies and start MLflow

import sys
import subprocess
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print(f"Python version: {sys.version}")
print("=" * 60)

# Core package verification
required_packages = {
    'numpy': 'Numerical computing',
    'pandas': 'Data manipulation', 
    'matplotlib': 'Basic plotting',
    'seaborn': 'Statistical visualization',
    'plotly': 'Interactive visualization',
    'sklearn': 'Machine learning',
    'librosa': 'Audio analysis',
    'soundfile': 'Audio I/O',
    'scipy': 'Scientific computing',
    'mlflow': 'Experiment tracking',
    'fastapi': 'Web API framework',
    'uvicorn': 'ASGI server',
    'requests': 'HTTP client',
    'jupyter': 'Jupyter ecosystem',
    'ipywidgets': 'Interactive widgets'
}

# Test TensorFlow and PyTorch separately
ml_frameworks = {
    'tensorflow': 'Deep learning framework (Google)',
    'torch': 'Deep learning framework (PyTorch)'
}

success_count = 0
total_count = len(required_packages) + len(ml_frameworks)

print("📦 Core Packages:")
for package, description in required_packages.items():
    try:
        if package == 'sklearn':
            import sklearn
        else:
            __import__(package)
        print(f"✅ {package:<15} - {description}")
        success_count += 1
    except ImportError:
        print(f"❌ {package:<15} - {description} (MISSING)")
        print(f"   Install with: pip install {package}")

print("\n🤖 ML Frameworks:")
for framework, description in ml_frameworks.items():
    try:
        if framework == 'tensorflow':
            import tensorflow as tf
            print(f"✅ {framework:<15} - {description} (v{tf.__version__})")
            # Test GPU availability
            if tf.config.list_physical_devices('GPU'):
                print(f"   🚀 GPU acceleration available: {len(tf.config.list_physical_devices('GPU'))} GPU(s)")
            else:
                print(f"   💻 CPU-only mode (no GPU detected)")
        elif framework == 'torch':
            import torch
            print(f"✅ {framework:<15} - {description} (v{torch.__version__})")
            if torch.cuda.is_available():
                print(f"   🚀 CUDA available: {torch.cuda.device_count()} GPU(s)")
            elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
                print(f"   🍎 Apple Silicon GPU (MPS) available")
            else:
                print(f"   💻 CPU-only mode")
        success_count += 1
    except ImportError:
        print(f"❌ {framework:<15} - {description} (MISSING)")
        print(f"   Install with: pip install {framework}")
    except Exception as e:
        print(f"⚠️  {framework:<15} - {description} (INSTALLED but issue: {str(e)[:50]}...)")
        success_count += 1  # Count as success since it's installed

print("=" * 60)
if success_count >= len(required_packages):  # Allow missing ML frameworks
    print(f"🎉 SUCCESS: Core packages ready! ({success_count}/{total_count} total packages)")
    print("You can proceed with the Orpheus Engine demo.")
else:
    missing = len(required_packages) + len(ml_frameworks) - success_count
    print(f"⚠️  WARNING: {missing} packages are missing.")
    print("Please install missing packages before continuing.")
    print("\nQuick fix: cd demo && pip install -r requirements.txt && pip install tensorflow torch")

In [ ]:
# 🚀 MLflow Startup and Configuration
# Initialize MLflow tracking for the Orpheus Engine demo

import mlflow
import mlflow.tensorflow
import mlflow.pytorch
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import subprocess
import time
import socket
from datetime import datetime
import threading
import os

def check_port_available(port):
    """Check if a port is available"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.bind(('127.0.0.1', port))
            return True
        except OSError:
            return False

def start_mlflow_server(port=5000):
    """Start MLflow tracking server if not already running"""
    if not check_port_available(port):
        print(f"✅ MLflow server already running on port {port}")
        return True
    
    try:
        # Create mlruns directory if it doesn't exist
        mlruns_dir = Path.cwd() / "mlruns"
        mlruns_dir.mkdir(exist_ok=True)
        
        print(f"🚀 Starting MLflow server on port {port}...")
        
        # Start MLflow server in background
        cmd = [
            sys.executable, "-m", "mlflow", "server",
            "--host", "127.0.0.1",
            "--port", str(port),
            "--backend-store-uri", f"sqlite:///{mlruns_dir}/mlflow.db",
            "--default-artifact-root", str(mlruns_dir)
        ]
        
        # Start process in background
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Wait a moment for server to start
        time.sleep(3)
        
        # Check if server is running
        if not check_port_available(port):
            print(f"✅ MLflow server started successfully on http://127.0.0.1:{port}")
            return True
        else:
            print(f"❌ Failed to start MLflow server")
            return False
            
    except Exception as e:
        print(f"❌ Error starting MLflow server: {e}")
        print("💡 You can start manually: mlflow server --host 127.0.0.1 --port 5000")
        return False

# MLflow Configuration
MLFLOW_PORT = 5000
MLFLOW_URI = f"http://127.0.0.1:{MLFLOW_PORT}"
EXPERIMENT_NAME = "Orpheus_Engine_Web_Demo"

print("🔧 MLflow Configuration:")
print(f"   Tracking URI: {MLFLOW_URI}")
print(f"   Experiment: {EXPERIMENT_NAME}")
print(f"   Artifacts: ./mlruns")
print()

# Start MLflow server
mlflow_running = start_mlflow_server(MLFLOW_PORT)

if mlflow_running:
    # Configure MLflow
    mlflow.set_tracking_uri(MLFLOW_URI)
    
    try:
        # Create or get experiment
        experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
        if experiment is None:
            experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
            print(f"✅ Created new experiment: {EXPERIMENT_NAME} (ID: {experiment_id})")
        else:
            experiment_id = experiment.experiment_id
            print(f"✅ Using existing experiment: {EXPERIMENT_NAME} (ID: {experiment_id})")
        
        mlflow.set_experiment(EXPERIMENT_NAME)
        
        # Test connection
        client = MlflowClient(MLFLOW_URI)
        experiments = client.search_experiments()
        print(f"✅ MLflow connection successful - Found {len(experiments)} experiments")
        
        print(f"\n🌐 MLflow UI available at: {MLFLOW_URI}")
        print("   Click the link above to view experiments in your browser")
        
    except Exception as e:
        print(f"⚠️  MLflow configuration issue: {e}")
        print("   Demo will continue without MLflow tracking")
else:
    print("⚠️  MLflow server not available - continuing without experiment tracking")
    print("   You can start MLflow manually and restart this notebook")

print("\n" + "="*60)
print("🎵 Orpheus Engine Demo Environment Ready!")
print("📊 Experiment tracking:", "✅ Enabled" if mlflow_running else "❌ Disabled")
print("🤖 ML Frameworks:", "TensorFlow + PyTorch" if 'tensorflow' in sys.modules and 'torch' in sys.modules else "Limited")
print("🏛️ Ready for DAW demonstrations and ML experiments")

In [ ]:
# Import required libraries and setup environment
import json
import os
import sys
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, Javascript, display, Audio
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic

# Add project root to path for importing Orpheus modules
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("🎵 Orpheus Engine Web Demo Environment Initialized")
print(f"📁 Project Root: {project_root}")
print(f"🐍 Python Version: {sys.version}")
print(f"📓 Jupyter Environment: {'JupyterLab' if 'JUPYTERHUB_SERVICE_PREFIX' in os.environ else 'Jupyter Notebook'}")

## 🔧 Platform Detection and Capabilities

Let's start by detecting the current platform and testing available capabilities:

In [ ]:
# Platform capability detection
def detect_platform_capabilities():
    """Detect and test platform capabilities for DAW functionality"""
    
    capabilities = {
        'python_backend': True,  # Always true in Jupyter
        'audio_processing': False,
        'file_system': True,
        'data_visualization': False,
        'machine_learning': False,
        'web_components': False
    }
    
    # Test audio processing libraries
    try:
        import librosa
        import soundfile as sf
        capabilities['audio_processing'] = True
        print("✅ Audio Processing: librosa, soundfile available")
    except ImportError:
        print("❌ Audio Processing: Install librosa and soundfile")
    
    # Test data visualization
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        capabilities['data_visualization'] = True
        print("✅ Data Visualization: matplotlib, seaborn available")
    except ImportError:
        print("❌ Data Visualization: Install matplotlib and seaborn")
    
    # Test machine learning capabilities
    try:
        import sklearn
        import pandas as pd
        capabilities['machine_learning'] = True
        print("✅ Machine Learning: scikit-learn, pandas available")
    except ImportError:
        print("❌ Machine Learning: Install scikit-learn and pandas")
    
    # Test web components support
    try:
        from IPython.display import HTML, Javascript
        capabilities['web_components'] = True
        print("✅ Web Components: IPython display available")
    except ImportError:
        print("❌ Web Components: IPython not available")
    
    return capabilities

# Run capability detection
platform_caps = detect_platform_capabilities()
print(f"\n📊 Platform Capabilities Summary:")
for cap, available in platform_caps.items():
    status = "✅" if available else "❌"
    print(f"{status} {cap.replace('_', ' ').title()}: {available}")

## 🎛️ Interactive DAW Components

Now let's create interactive web components that simulate DAW functionality:

In [ ]:
def create_daw_interface():
    """Create an interactive DAW interface using HTML/CSS/JavaScript"""
    
    daw_html = """
    <div id="orpheus-daw-demo" style="
        width: 100%; 
        background: linear-gradient(135deg, #1e1e1e 0%, #2a2a2a 100%); 
        color: white; 
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
        border-radius: 12px; 
        padding: 20px; 
        margin: 10px 0;
        box-shadow: 0 4px 20px rgba(0,0,0,0.3);
    ">
        <div style="text-align: center; margin-bottom: 30px;">
            <h1 style="
                background: linear-gradient(45deg, #ff6b6b, #4ecdc4); 
                -webkit-background-clip: text; 
                -webkit-text-fill-color: transparent; 
                background-clip: text; 
                margin: 0; 
                font-size: 2.5rem;
            ">Orpheus Engine Demo</h1>
            <p style="color: #888; margin: 10px 0;">Interactive DAW Components</p>
        </div>
        
        <!-- Platform Info Panel -->
        <div style="background: rgba(0,0,0,0.3); border-radius: 8px; padding: 16px; margin-bottom: 20px;">
            <h3 style="color: #4ecdc4; margin-top: 0;">Platform Information</h3>
            <div id="platform-info" style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 10px;">
                <div style="background: rgba(78, 205, 196, 0.2); padding: 10px; border-radius: 6px; border: 1px solid #4ecdc4;">
                    <strong>Environment:</strong> Jupyter Notebook
                </div>
                <div style="background: rgba(78, 205, 196, 0.2); padding: 10px; border-radius: 6px; border: 1px solid #4ecdc4;">
                    <strong>Backend:</strong> Python
                </div>
                <div style="background: rgba(78, 205, 196, 0.2); padding: 10px; border-radius: 6px; border: 1px solid #4ecdc4;">
                    <strong>Frontend:</strong> HTML/JS
                </div>
            </div>
        </div>
        
        <!-- Transport Controls -->
        <div style="background: rgba(0,0,0,0.3); border-radius: 8px; padding: 16px; margin-bottom: 20px;">
            <h3 style="color: #4ecdc4; margin-top: 0;">Transport Controls</h3>
            <div style="display: flex; align-items: center; gap: 12px; margin-bottom: 16px;">
                <button onclick="togglePlayback()" id="play-btn" style="
                    width: 50px; height: 50px; border-radius: 50%; background: #4ecdc4; 
                    border: none; color: #1e1e1e; font-size: 1.5rem; cursor: pointer;
                    transition: all 0.2s;
                " onmouseover="this.style.background='#45b7aa'" onmouseout="this.style.background='#4ecdc4'">
                    ▶️
                </button>
                <button onclick="stopPlayback()" style="
                    width: 40px; height: 40px; border-radius: 6px; background: #ff6b6b; 
                    border: none; color: white; font-size: 1.2rem; cursor: pointer;
                ">⏹️</button>
                <span id="time-display" style="
                    font-family: monospace; background: rgba(0,0,0,0.5); 
                    padding: 8px 16px; border-radius: 4px; min-width: 100px; text-align: center;
                ">00:00.0</span>
                <div style="flex: 1; height: 6px; background: #333; border-radius: 3px; position: relative;">
                    <div id="progress-bar" style="
                        height: 100%; background: #4ecdc4; border-radius: 3px; width: 0%; 
                        transition: width 0.1s;
                    "></div>
                </div>
            </div>
        </div>
        
        <!-- Timeline/Tracks -->
        <div style="background: rgba(0,0,0,0.3); border-radius: 8px; padding: 16px; margin-bottom: 20px;">
            <h3 style="color: #4ecdc4; margin-top: 0;">Timeline & Tracks</h3>
            <div style="background: #2a2a2a; border-radius: 6px; padding: 12px; min-height: 160px; position: relative;">
                <div id="playhead" style="
                    position: absolute; top: 0; width: 2px; height: 100%; 
                    background: #ff6b6b; z-index: 10; left: 0; transition: left 0.1s;
                "></div>
                
                <!-- Track 1 -->
                <div style="height: 40px; border-bottom: 1px solid #444; position: relative; margin-bottom: 8px;">
                    <div style="
                        position: absolute; left: 10%; width: 25%; height: 30px; top: 5px;
                        background: linear-gradient(45deg, #4ecdc4, #45b7aa); border-radius: 4px;
                        display: flex; align-items: center; justify-content: center; font-size: 0.8rem;
                    ">Kick Drum</div>
                    <span style="position: absolute; left: 8px; top: 50%; transform: translateY(-50%); font-size: 0.9rem;">
                        Track 1
                    </span>
                </div>
                
                <!-- Track 2 -->
                <div style="height: 40px; border-bottom: 1px solid #444; position: relative; margin-bottom: 8px;">
                    <div style="
                        position: absolute; left: 20%; width: 35%; height: 30px; top: 5px;
                        background: linear-gradient(45deg, #ff6b6b, #ff5555); border-radius: 4px;
                        display: flex; align-items: center; justify-content: center; font-size: 0.8rem;
                    ">Bass Synth</div>
                    <span style="position: absolute; left: 8px; top: 50%; transform: translateY(-50%); font-size: 0.9rem;">
                        Track 2
                    </span>
                </div>
                
                <!-- Track 3 -->
                <div style="height: 40px; position: relative;">
                    <div style="
                        position: absolute; left: 40%; width: 40%; height: 30px; top: 5px;
                        background: linear-gradient(45deg, #9b59b6, #8e44ad); border-radius: 4px;
                        display: flex; align-items: center; justify-content: center; font-size: 0.8rem;
                    ">Lead Vocal</div>
                    <span style="position: absolute; left: 8px; top: 50%; transform: translateY(-50%); font-size: 0.9rem;">
                        Track 3
                    </span>
                </div>
            </div>
        </div>
        
        <!-- Mixer Panel -->
        <div style="background: rgba(0,0,0,0.3); border-radius: 8px; padding: 16px;">
            <h3 style="color: #4ecdc4; margin-top: 0;">Mixer</h3>
            <div style="display: flex; justify-content: space-around; align-items: end;">
                <!-- Channel 1 -->
                <div style="display: flex; flex-direction: column; align-items: center; gap: 10px;">
                    <div id="meter-1" style="width: 20px; height: 100px; background: #333; border-radius: 10px; position: relative; overflow: hidden;">
                        <div style="position: absolute; bottom: 0; width: 100%; background: linear-gradient(to top, #4ecdc4, #ff6b6b); height: 0%; transition: height 0.1s;" id="meter-1-level"></div>
                    </div>
                    <input type="range" min="0" max="100" value="75" style="writing-mode: bt-lr; -webkit-appearance: slider-vertical; width: 20px; height: 80px;" oninput="updateChannelVolume(1, this.value)">
                    <span style="font-size: 0.8rem;">Ch 1</span>
                </div>
                
                <!-- Channel 2 -->
                <div style="display: flex; flex-direction: column; align-items: center; gap: 10px;">
                    <div id="meter-2" style="width: 20px; height: 100px; background: #333; border-radius: 10px; position: relative; overflow: hidden;">
                        <div style="position: absolute; bottom: 0; width: 100%; background: linear-gradient(to top, #4ecdc4, #ff6b6b); height: 0%; transition: height 0.1s;" id="meter-2-level"></div>
                    </div>
                    <input type="range" min="0" max="100" value="68" style="writing-mode: bt-lr; -webkit-appearance: slider-vertical; width: 20px; height: 80px;" oninput="updateChannelVolume(2, this.value)">
                    <span style="font-size: 0.8rem;">Ch 2</span>
                </div>
                
                <!-- Channel 3 -->
                <div style="display: flex; flex-direction: column; align-items: center; gap: 10px;">
                    <div id="meter-3" style="width: 20px; height: 100px; background: #333; border-radius: 10px; position: relative; overflow: hidden;">
                        <div style="position: absolute; bottom: 0; width: 100%; background: linear-gradient(to top, #4ecdc4, #ff6b6b); height: 0%; transition: height 0.1s;" id="meter-3-level"></div>
                    </div>
                    <input type="range" min="0" max="100" value="82" style="writing-mode: bt-lr; -webkit-appearance: slider-vertical; width: 20px; height: 80px;" oninput="updateChannelVolume(3, this.value)">
                    <span style="font-size: 0.8rem;">Ch 3</span>
                </div>
                
                <!-- Master -->
                <div style="display: flex; flex-direction: column; align-items: center; gap: 10px; border-left: 2px solid #444; padding-left: 20px; margin-left: 15px;">
                    <div id="master-meter" style="width: 25px; height: 100px; background: #333; border-radius: 10px; position: relative; overflow: hidden;">
                        <div style="position: absolute; bottom: 0; width: 100%; background: linear-gradient(to top, #4ecdc4, #ff6b6b); height: 0%; transition: height 0.1s;" id="master-meter-level"></div>
                    </div>
                    <input type="range" min="0" max="100" value="85" style="writing-mode: bt-lr; -webkit-appearance: slider-vertical; width: 25px; height: 80px;" oninput="updateMasterVolume(this.value)">
                    <span style="font-size: 0.9rem; font-weight: bold;">Master</span>
                </div>
            </div>
        </div>
    </div>
    
    <script>
        let isPlaying = false;
        let currentTime = 0;
        let playInterval = null;
        let meterInterval = null;
        
        function togglePlayback() {
            const playBtn = document.getElementById('play-btn');
            const timeDisplay = document.getElementById('time-display');
            const progressBar = document.getElementById('progress-bar');
            const playhead = document.getElementById('playhead');
            
            if (isPlaying) {
                // Stop playback
                isPlaying = false;
                playBtn.innerHTML = '▶️';
                clearInterval(playInterval);
                clearInterval(meterInterval);
            } else {
                // Start playback
                isPlaying = true;
                playBtn.innerHTML = '⏸️';
                
                // Update time and progress
                playInterval = setInterval(() => {
                    currentTime += 0.1;
                    const minutes = Math.floor(currentTime / 60);
                    const seconds = (currentTime % 60).toFixed(1);
                    timeDisplay.textContent = `${minutes.toString().padStart(2, '0')}:${seconds.padStart(4, '0')}`;
                    
                    // Update progress bar (assuming 30 second loop)
                    const progress = (currentTime % 30) / 30 * 100;
                    progressBar.style.width = progress + '%';
                    playhead.style.left = progress + '%';
                    
                    // Loop back to start after 30 seconds
                    if (currentTime >= 30) currentTime = 0;
                }, 100);
                
                // Animate meters
                meterInterval = setInterval(() => {
                    for (let i = 1; i <= 3; i++) {
                        const level = Math.random() * 70 + 20; // 20-90%
                        document.getElementById(`meter-${i}-level`).style.height = level + '%';
                    }
                    const masterLevel = Math.random() * 80 + 15; // 15-95%
                    document.getElementById('master-meter-level').style.height = masterLevel + '%';
                }, 50);
            }
        }
        
        function stopPlayback() {
            isPlaying = false;
            currentTime = 0;
            document.getElementById('play-btn').innerHTML = '▶️';
            document.getElementById('time-display').textContent = '00:00.0';
            document.getElementById('progress-bar').style.width = '0%';
            document.getElementById('playhead').style.left = '0%';
            clearInterval(playInterval);
            clearInterval(meterInterval);
            
            // Reset meters
            for (let i = 1; i <= 3; i++) {
                document.getElementById(`meter-${i}-level`).style.height = '0%';
            }
            document.getElementById('master-meter-level').style.height = '0%';
        }
        
        function updateChannelVolume(channel, value) {
            console.log(`Channel ${channel} volume: ${value}%`);
        }
        
        function updateMasterVolume(value) {
            console.log(`Master volume: ${value}%`);
        }
    </script>
    """
    
    return HTML(daw_html)

# Display the DAW interface
create_daw_interface()

## 🎵 Audio Processing Demo

Let's demonstrate audio processing capabilities using Python libraries:

In [ ]:
# Audio processing and waveform generation
def create_audio_demo():
    """Create audio waveforms and demonstrate processing"""
    
    if not platform_caps.get('audio_processing', False):
        print("❌ Audio processing libraries not available. Install librosa and soundfile:")
        print("pip install librosa soundfile")
        return
    
    # Generate test audio signals
    sample_rate = 44100
    duration = 3.0  # seconds
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # Create different waveforms
    kick_drum = np.sin(2 * np.pi * 60 * t) * np.exp(-t * 10) * 0.8  # Exponential decay
    bass_synth = np.sin(2 * np.pi * 80 * t) * 0.6  # Sine wave
    lead_vocal = np.sin(2 * np.pi * 220 * t + np.sin(2 * np.pi * 5 * t)) * 0.4  # FM synthesis
    
    # Mix the signals
    mix = (kick_drum + bass_synth + lead_vocal) / 3
    
    # Apply some basic effects
    # Simple low-pass filter
    from scipy import signal
    b, a = signal.butter(4, 0.1)
    filtered_mix = signal.filtfilt(b, a, mix)
    
    # Plot waveforms
    plt.figure(figsize=(15, 10))
    
    plt.subplot(3, 2, 1)
    plt.plot(t[:1000], kick_drum[:1000], color='#4ecdc4', linewidth=2)
    plt.title('Track 1: Kick Drum', fontsize=12, color='#4ecdc4')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 2, 2)
    plt.plot(t[:1000], bass_synth[:1000], color='#ff6b6b', linewidth=2)
    plt.title('Track 2: Bass Synth', fontsize=12, color='#ff6b6b')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 2, 3)
    plt.plot(t[:1000], lead_vocal[:1000], color='#9b59b6', linewidth=2)
    plt.title('Track 3: Lead Vocal', fontsize=12, color='#9b59b6')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 2, 4)
    plt.plot(t[:1000], mix[:1000], color='#f39c12', linewidth=2)
    plt.title('Mixed Output', fontsize=12, color='#f39c12')
    plt.grid(True, alpha=0.3)
    
    # Frequency analysis
    plt.subplot(3, 2, 5)
    f, Pxx = signal.welch(mix, sample_rate, nperseg=2048)
    plt.semilogy(f, Pxx, color='#e74c3c', linewidth=2)
    plt.title('Frequency Spectrum', fontsize=12, color='#e74c3c')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 1000)
    
    # Filtered version
    plt.subplot(3, 2, 6)
    plt.plot(t[:1000], filtered_mix[:1000], color='#27ae60', linewidth=2)
    plt.title('Filtered Mix (Low-pass)', fontsize=12, color='#27ae60')
    plt.xlabel('Time (s)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Return audio for playback
    return {
        'sample_rate': sample_rate,
        'kick_drum': kick_drum,
        'bass_synth': bass_synth,
        'lead_vocal': lead_vocal,
        'mix': mix,
        'filtered_mix': filtered_mix
    }

# Generate and display audio demo
audio_data = create_audio_demo()

## 🎧 Audio Playback

Listen to the generated audio tracks:

In [ ]:
# Audio playback functionality
if audio_data and platform_caps.get('audio_processing', False):
    print("🎵 Generated Audio Tracks:")
    print("Click on any audio player below to listen to the generated sounds")
    
    # Display audio players for each track
    display(HTML("<h4 style='color: #4ecdc4;'>🥁 Track 1: Kick Drum</h4>"))
    display(Audio(audio_data['kick_drum'], rate=audio_data['sample_rate']))
    
    display(HTML("<h4 style='color: #ff6b6b;'>🎹 Track 2: Bass Synth</h4>"))
    display(Audio(audio_data['bass_synth'], rate=audio_data['sample_rate']))
    
    display(HTML("<h4 style='color: #9b59b6;'>🎤 Track 3: Lead Vocal</h4>"))
    display(Audio(audio_data['lead_vocal'], rate=audio_data['sample_rate']))
    
    display(HTML("<h4 style='color: #f39c12;'>🎛️ Mixed Output</h4>"))
    display(Audio(audio_data['mix'], rate=audio_data['sample_rate']))
    
    display(HTML("<h4 style='color: #27ae60;'>🔊 Filtered Mix</h4>"))
    display(Audio(audio_data['filtered_mix'], rate=audio_data['sample_rate']))
    
else:
    print("❌ Audio playback requires audio processing libraries")
    print("Install with: pip install librosa soundfile")

## 📊 Platform Integration Test

Test integration between the web interface and Python backend:

In [ ]:
# Platform integration testing
def test_platform_integration():
    """Test integration between frontend and backend components"""
    
    print("🔄 Testing Platform Integration...")
    print("=" * 50)
    
    # Test 1: Data sharing between Python and JavaScript
    test_data = {
        'timestamp': datetime.now().isoformat(),
        'tracks': [
            {'id': 1, 'name': 'Kick Drum', 'volume': 0.8, 'muted': False},
            {'id': 2, 'name': 'Bass Synth', 'volume': 0.7, 'muted': False},
            {'id': 3, 'name': 'Lead Vocal', 'volume': 0.9, 'muted': False}
        ],
        'master_volume': 0.85,
        'sample_rate': 44100,
        'project_length': 30.0
    }
    
    print("✅ Test 1: Data Structure Creation")
    print(f"   Generated project data with {len(test_data['tracks'])} tracks")
    
    # Test 2: JSON serialization (for web component communication)
    try:
        json_data = json.dumps(test_data, indent=2)
        print("✅ Test 2: JSON Serialization")
        print("   Data successfully serialized for web component communication")
    except Exception as e:
        print(f"❌ Test 2: JSON Serialization Failed - {e}")
    
    # Test 3: File system operations
    try:
        demo_path = Path("demo_session.json")
        with open(demo_path, 'w') as f:
            json.dump(test_data, f, indent=2)
        print("✅ Test 3: File System Write")
        print(f"   Session data written to {demo_path}")
        
        # Clean up
        demo_path.unlink()
    except Exception as e:
        print(f"❌ Test 3: File System Write Failed - {e}")
    
    # Test 4: Audio processing integration
    if platform_caps.get('audio_processing', False):
        try:
            # Create a simple audio buffer
            audio_buffer = np.random.random(1024) * 0.1
            print("✅ Test 4: Audio Buffer Creation")
            print(f"   Created audio buffer with {len(audio_buffer)} samples")
        except Exception as e:
            print(f"❌ Test 4: Audio Buffer Creation Failed - {e}")
    else:
        print("⚠️  Test 4: Audio Processing Skipped (libraries not available)")
    
    # Test 5: Web component state management
    component_state = {
        'transport': {
            'playing': False,
            'position': 0.0,
            'loop_enabled': True,
            'tempo': 120
        },
        'mixer': {
            'channels': [
                {'volume': 0.8, 'pan': 0.0, 'mute': False, 'solo': False},
                {'volume': 0.7, 'pan': -0.2, 'mute': False, 'solo': False},
                {'volume': 0.9, 'pan': 0.1, 'mute': False, 'solo': False}
            ],
            'master': {'volume': 0.85, 'mute': False}
        }
    }
    
    print("✅ Test 5: Component State Management")
    print(f"   Created state for {len(component_state)} component groups")
    
    print("\n🎯 Integration Test Summary:")
    print("   ✅ Python Backend: Ready")
    print("   ✅ Web Components: Ready")
    print("   ✅ Data Exchange: Working")
    print("   ✅ State Management: Working")
    print(f"   {'✅' if platform_caps.get('audio_processing') else '⚠️ '} Audio Processing: {'Ready' if platform_caps.get('audio_processing') else 'Limited'}")
    
    return {
        'status': 'success',
        'capabilities': platform_caps,
        'test_data': test_data,
        'component_state': component_state
    }

# Run integration tests
integration_results = test_platform_integration()

## 🚀 Export and Sharing

Export your demo session and component configurations:

In [ ]:
# Export demo configuration and results
def export_demo_session():
    """Export the complete demo session for sharing or further development"""
    
    export_data = {
        'session_info': {
            'timestamp': datetime.now().isoformat(),
            'platform': 'Jupyter Notebook',
            'environment': 'Python Backend',
            'capabilities': platform_caps
        },
        'components_tested': [
            'Transport Controls',
            'Timeline Interface', 
            'Mixer Panel',
            'Audio Processing',
            'Platform Integration'
        ],
        'audio_generated': bool(audio_data) if 'audio_data' in globals() else False,
        'integration_results': integration_results,
        'recommendations': []
    }
    
    # Add recommendations based on capabilities
    if not platform_caps.get('audio_processing'):
        export_data['recommendations'].append('Install audio processing libraries: pip install librosa soundfile')
    
    if platform_caps.get('machine_learning'):
        export_data['recommendations'].append('Consider adding ML-based audio analysis features')
    
    if platform_caps.get('web_components'):
        export_data['recommendations'].append('Web components ready for advanced interactive features')
    
    # Save to file
    export_path = Path("orpheus_demo_session.json")
    with open(export_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print("📄 Demo Session Export Complete")
    print(f"📁 Saved to: {export_path}")
    print(f"📊 Session includes {len(export_data['components_tested'])} tested components")
    print(f"💡 {len(export_data['recommendations'])} recommendations generated")
    
    # Display summary
    display(HTML(f"""
    <div style="background: #2c3e50; color: white; padding: 20px; border-radius: 8px; margin: 10px 0;">
        <h3 style="color: #3498db;">📋 Demo Session Summary</h3>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px;">
            <div>
                <strong>Environment:</strong><br>
                {export_data['session_info']['platform']}<br>
                {export_data['session_info']['environment']}
            </div>
            <div>
                <strong>Components Tested:</strong><br>
                {len(export_data['components_tested'])} components
            </div>
            <div>
                <strong>Capabilities:</strong><br>
                {sum(export_data['session_info']['capabilities'].values())} / {len(export_data['session_info']['capabilities'])} available
            </div>
            <div>
                <strong>Audio Generated:</strong><br>
                {'Yes' if export_data['audio_generated'] else 'No'}
            </div>
        </div>
        <div style="margin-top: 15px;">
            <strong>Next Steps:</strong>
            <ul>
                {''.join(f'<li>{rec}</li>' for rec in export_data['recommendations']) if export_data['recommendations'] else '<li>All capabilities available - ready for production!</li>'}
            </ul>
        </div>
    </div>
    """))
    
    return export_data

# Export the session
session_export = export_demo_session()

## 🎯 Next Steps

This demo showcases the Orpheus Engine's cross-platform capabilities. Here's what you can do next:

### 🔧 Development
- **Integrate with Electron**: Use the same components in the desktop application
- **Add Real Audio**: Connect with professional audio interfaces
- **Extend Components**: Add more DAW features like effects, synthesis, and recording

### 🚀 Deployment
- **Web Version**: Deploy components as web applications
- **Desktop App**: Package with Electron for native desktop experience  
- **Backend Integration**: Connect with Python audio processing backend

### 📚 Documentation
- **API Reference**: Document component interfaces and capabilities
- **User Guide**: Create tutorials for different user types
- **Developer Guide**: Setup instructions for contributors

The demo files are saved in your working directory and can be imported into other projects or shared with team members.